
## INE Instituto nacional de estadística

In [1]:
%load_ext autoreload
%autoreload 2
import json
import os
import re
import sys
from datetime import date
from pathlib import Path

import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import pandas as pd
import requests

cwd = Path.cwd()
home = Path.home()
module_path = f"{home}/serhi/src/lib" if cwd == home else f"{cwd.parents[1]}/lib"
sys.path.append(module_path)
import serhi

/home/node /home/node True
/home/node/serhi/src/lib
SerHi Tools


In [3]:
# inicio
# cambiar como corresponda
ENTITY = "ine"
NB_NAME = "ine_ipri"
serhi.initialize(ENTITY, NB_NAME)

serhi.CWD /home/node
serhi.BASE_PATH ../../..
serhi.DATA_PATH ../../../data/process/ine/
serhi.DOWN_PATH ../../../tmp/download/ine/
serhi.OUT_FILE ../../../data/process/ine/ine_ipri.csv


PermissionError: [Errno 13] Permission denied: '../../../data'

In [ ]:
print(dir(serhi))

In [ ]:
def download_files(urls):
    for loc in urls:
        url = loc["url"]
        r = requests.get(url, allow_redirects=True)
        print(f"Downloading {url}")
        open(serhi.DOWN_PATH + loc["file"], "wb").write(r.content)


urls = [
    # ipri
    {
        "url": "https://www.ine.es/jaxiT3/files/t/es/xlsx/27065.xlsx?nocab=1",
        "file": "ine_ipri.xlsx",
    },
]

download_files(urls)

In [ ]:
# IPRI
loc = urls[0]
file = loc["file"]
df = pd.read_excel(serhi.DOWN_PATH + file, skiprows=6, nrows=162, header=(0, 1))
df.head()

In [ ]:
df.tail()

In [ ]:
df.columns.levels[0]

In [ ]:
df = df.rename(columns={" ": "Name"})

In [ ]:
# da = df.xs('Índice', axis=1, level=0, drop_level=True)

In [ ]:
df = df[['Name', 'Índice']]

In [ ]:
df

In [ ]:
df.columns = df.columns.droplevel()
df

In [ ]:
def ine_sheet_ccaa(df, set_name, nccaa=18, ndatos=9, ud="idx"):
    dr = pd.DataFrame()
    for i in range(0, nccaa):
        # get next rows
        dt = df.loc[i * ndatos : i * ndatos + (ndatos - 1)]
        dt.reset_index(inplace=True, drop=True)
        # save zone
        zone = dt.loc[0][0]
        # delete first row
        dt = dt.loc[1:]
        # change columns names - to months
        columns = [w.replace("M", "-") for w in dt.columns]
        columns = [re.sub(r"\.[0-9]", "", w) for w in columns]
        columns = [w + "-01" for w in columns]
        columns[0] = "Name"
        dt.columns = columns
        dt = dt.T
        # promote columns
        dt.columns = dt.iloc[0]
        dt = dt[1:]
        dt.columns = [w.strip() for w in dt.columns]
        # change index name
        dt.index.rename("Date", inplace=True)
        # sort and scale
        dt.sort_index(ascending=True, inplace=True)
        # transpose
        dt = dt.unstack()
        dt = dt.reset_index()
        # set zone
        dt["Zone"] = zone
        dt["Ud"] = ud
        # set column value
        dt = dt.rename(columns={0: "Value"})
        # concat
        dr = pd.concat([dr, dt])

    dr["Set"] = set_name
    dr.rename(columns={"level_0": "Name"}, inplace=True)
    return dr

In [ ]:
df = ine_sheet_ccaa(df, set_name="INE_IPRI")
df.head()

In [ ]:
df.info()

In [ ]:
df["Name"].unique()

In [ ]:
df["Zone"].unique()

In [ ]:
df

In [ ]:
df["Date"] = pd.to_datetime(df["Date"])
df["Date"].agg(["min", "max"])

In [ ]:
df.info()

In [ ]:
# fill na with 0
df["Value"] = pd.to_numeric(df["Value"], errors="coerce")
df["Value"] = df["Value"].fillna(0)

In [ ]:
names = df["Name"].unique()
zones = df["Zone"].unique()

for z in zones:
    fig, ax = plt.subplots(figsize=(15, 8))
    print(z)
    dx = df[df["Zone"] == z]
    plt.title(z)
    for n in names:
        dxz = dx[dx["Name"] == n]
        plt.plot(dxz["Date"], dxz["Value"], label=f"{n}")
    plt.legend(title="Group")
    plt.setp(ax.get_xticklabels(), rotation=90)
    plt.show()

In [ ]:
# Guardamos la serie procesada
df.to_csv(serhi.OUT_FILE, index=False)